In [ ]:
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
df = pd.read_csv(r'..\BADENOVAnetze Daten\Strom-Einspeiser-Export 1.csv', encoding='ansi', sep=';', on_bad_lines='skip')

In [ ]:
# Entferne überschüssige Spalten 'Anzahl_Haeuser_x' und 'Anzahl_Haeuser_y'
df = df.drop(columns=['Anzahl_Haeuser_x', 'Anzahl_Haeuser_y'], errors='ignore')

# Datum parsen
df['Einbaudatum'] = pd.to_datetime(df['Einbaudatum'], errors='coerce', dayfirst=True)

# Nur gültige Datumszeilen behalten
df = df[df['Einbaudatum'].notnull()]  

# Jahr und Monat extrahieren
df['Jahr'] = df['Einbaudatum'].dt.year
df['Monat'] = df['Einbaudatum'].dt.month

In [ ]:
# Dictionary mit Gebäudeanzahl
anzahl_gebaeude = {
    "Freiburg": 32249,
    "Gundelfingen": 2686,
    "Kirchzarten": 2260,
    "Bad Krozingen": 4829,
    "Breisach": 3671,
    "Buggingen": 985,
    "Ebringen": 646,
    "Gottenheim": 687,
    "Heitersheim": 1430,
    "Ihringen": 1404,
    "Lauchringen": 1790,
    "Merdingen": 590,
    "Müllheim": 4437,
    "Neuenburg": 2839,
    "St. Peter": 608,
    "Staufen": 1884,
    "Stegen": 997,
    "Titisee-Neustadt": 2659,
    "Umkirch": 1108,
    "Weisweil": 466,
    "Wutöschingen": 1330
}

# In DataFrame umwandeln
gebaeude_df = pd.DataFrame(list(anzahl_gebaeude.items()), columns=["Gemeinde", "Anzahl_Haeuser"])

# Mit df mergen
df = df.merge(gebaeude_df, on="Gemeinde", how="left")


In [ ]:
df = pd.read_csv(r'..\BADENOVAnetze Daten\Strom-Einspeiser-Export 1.csv', encoding='ansi', sep=';', on_bad_lines='skip')

In [ ]:
# Stelle sicher, dass Einbaudatum im datetime-Format ist
df['Einbaudatum'] = pd.to_datetime(df['Einbaudatum'], errors='coerce', dayfirst=True)
df = df[df['Einbaudatum'].notnull()]  # Ungültige Daten entfernen

# Jahr und Monat extrahieren
df['Jahr'] = df['Einbaudatum'].dt.year
df['Monat'] = df['Einbaudatum'].dt.month

# Gruppieren nach Gemeinde, Jahr und Monat
gruppen = df.groupby(['Gemeinde', 'Jahr', 'Monat']).size().reset_index(name='Anzahl')

# Datumsspalte zum Plotten erstellen
gruppen['Datum'] = pd.to_datetime(
    gruppen['Jahr'].astype(str) + '-' +
    gruppen['Monat'].astype(str).str.zfill(2) + '-01'
)

# Filter auf Daten ab 2001
gruppen = gruppen[gruppen['Datum'].dt.year >= 2001]

# Optional: Nur Top 21 Gemeinden nach Häufigkeit
top_gemeinden = df['Gemeinde'].value_counts().index
gruppen = gruppen[gruppen['Gemeinde'].isin(top_gemeinden)]

# Gesamtanzahl Häuser je Gemeinde hinzufügen
gruppen = gruppen.merge(gebaeude_df, on="Gemeinde", how="left")

# Kumulierte Anzahl berechnen
gruppen['Kumuliert'] = gruppen.groupby('Gemeinde')['Anzahl'].cumsum()

# Prozentualer Anteil
gruppen['Kumuliert_Prozent'] = (gruppen['Kumuliert'] / gruppen['Anzahl_Haeuser']) * 100

# Plot
plt.figure(figsize=(14, 7))

for gemeinde, gruppe in gruppen.groupby('Gemeinde'):
    plt.plot(gruppe['Datum'], gruppe['Kumuliert_Prozent'], label=gemeinde)

plt.title('Kumulierte Einbauten in % pro Monat nach Gemeinde (ab 2001)')
plt.xlabel('Monat')
plt.ylabel('Kumulierte Einbauten in % der Gebäude')
plt.legend(loc='upper left', fontsize='small', ncol=2)
plt.grid(True)
plt.xticks(rotation=45)
plt.xlim(pd.to_datetime('2001-01-01'), pd.to_datetime('2024-12-31'))
plt.tight_layout()
plt.show()


## 2000–2003: 100.000-Dächer-Programm

Dieses Programm bot zinsgünstige Kredite für die Installation von Photovoltaikanlagen und war ein bedeutender Anreiz für Privatpersonen und kleine Unternehmen. Es endete 2003, nachdem das Ziel von 300 MW installierter Leistung erreicht wurde. 
Wikipedia

## Ab 2000: Erneuerbare-Energien-Gesetz (EEG)

Das EEG garantierte feste Einspeisevergütungen über 20 Jahre für eingespeisten Solarstrom, was Investitionssicherheit schuf. Die Vergütungssätze wurden jedoch regelmäßig angepasst, um Marktveränderungen zu berücksichtigen. 
FNP
+4
Grünes Haus
+4
Wikipedia
+4
Grünes Haus
+2
FNP
+2
Wikipedia
+2

## 2012: Einführung des 52-GW-Deckels

Eine Obergrenze von 52 GW förderbarer Solarleistung wurde eingeführt, was zu einem Rückgang des Zubaus führte. Diese Begrenzung wurde 2020 aufgehoben, um den weiteren Ausbau zu ermöglichen. 
Wikipedia

## 2022: Osterpaket

Mit dem Osterpaket wurden Änderungen im EEG beschlossen, darunter die Abschaffung der 70%-Regelung für Neuanlagen bis 25 kW und eine Anpassung der Degressionsmechanismen für die Einspeisevergütung. 
FNP
+3
BSH Energie
+3
Wikipedia
+3

## 2023: Nullsteuersatz für Photovoltaik

Seit dem 1. Januar 2023 gilt ein Nullsteuersatz für Photovoltaikanlagen bis 30 kWp, wodurch keine Umsatzsteuer auf Kauf und Installation anfällt. Dies senkte die Anschaffungskosten erheblich. 
G